In [169]:
import argparse
from uuid import uuid4
import pandas as pd

In [170]:
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.json_schema import JSONSerializer

In [171]:
from configparser import ConfigParser
configure=ConfigParser()
configure.read('kafka_config.ini')

['kafka_config.ini']

In [172]:
configure['Schema_Registry']['ENDPOINT_SCHEMA_URL']

'https://psrc-10dzz.ap-southeast-2.aws.confluent.cloud'

In [173]:
def sasl_conf():

    sasl_conf = {'sasl.mechanism': configure['Security_Params']['SSL_MACHENISM'],
                'bootstrap.servers':configure['Kafka_Cluster_Config']['BOOTSTRAP_SERVER'],
                'security.protocol': configure['Security_Params']['SECURITY_PROTOCOL'],
                'sasl.username': configure['Kafka_Cluster_Config']['API_KEY'],
                'sasl.password': configure['Kafka_Cluster_Config']['API_SECRET_KEY']
                }
    return sasl_conf

print(sasl_conf())

{'sasl.mechanism': 'PLAIN', 'bootstrap.servers': 'pkc-l7pr2.ap-south-1.aws.confluent.cloud:9092', 'security.protocol': 'SASL_SSL', 'sasl.username': 'QLCYOZMMI4FACXRR', 'sasl.password': '6QLw8EsONWzUd9QxcWJj17GSDwkdtEZCfAl4XgLIiT/Tlr+ATu01/pbhL1kDwmBb'}


In [174]:
def schema_config():
    return {'url':configure['Schema_Registry']['ENDPOINT_SCHEMA_URL'],
    'basic.auth.user.info':f"{configure['Schema_Registry']['SCHEMA_REGISTRY_API_KEY']}:{configure['Schema_Registry']['SCHEMA_REGISTRY_API_SECRET']}"

    }

print(schema_config())

{'url': 'https://psrc-10dzz.ap-southeast-2.aws.confluent.cloud', 'basic.auth.user.info': 'BADBMWX2ENBDFSPQ:/oGF2gx7Y+2nTertWRL6snJ3NEMR6lMtDCBrcVH1M/FJ6gHHBzqK7Jowbh+o3wcD'}


In [175]:
import importlib
import dbutils
import datetime
importlib.reload(dbutils)
from dbutils import Dbutils as db

prev_date=datetime.datetime(2024,1,8)

In [176]:
class Car:   
    def __init__(self,record:dict):
        for k,v in record.items():
            setattr(self,k,v)
        
        self.record=record
   
    @staticmethod
    def dict_to_car(data:dict,ctx):
        return Car(record=data)

    def __str__(self):
        return f"{self.record}"

In [177]:
def car_to_dict(car:Car, ctx):
    """
    Returns a dict representation of a User instance for serialization.
    Args:
        user (User): User instance.
        ctx (SerializationContext): Metadata pertaining to the serialization
            operation.
    Returns:
        dict: Dict populated with user attributes to be serialized.
    """

    # User._address must not be serialized; omit from dict
    return car.record

def delivery_report(err, msg):
    """
    Reports the success or failure of a message delivery.
    Args:
        err (KafkaError): The error that occurred on None on success.
        msg (Message): The message that was produced or failed.
    """

    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [178]:
db_obj=db()
db_obj.db_connect()

def get_car_instance():
    query="select max(Record_ld_dt) as max_record_ld_dts from kafka_test.cardekho_sales_dtl"
    max_date=db_obj.run_query(query)['max_record_ld_dts'].values[0]
    global prev_date
    if max_date==prev_date:
        print("No New data found")
        return -1
    else:
        query=f"select * from kafka_test.cardekho_sales_dtl where Record_ld_dt='{max_date}'"
        df=db_obj.run_query(query)
        df['Record_ld_dt']=df['Record_ld_dt'].astype(str)
        prev_date=max_date
        for data in df.values:
            car=Car(dict(zip(df.columns,data)))
            yield car
    

In [102]:
df.head()

,Idx,Car_name,Brand,Model,Vehicle_age,Km_driven,Seller_type,Fuel_type,Transmission_type,Mileage,Engine,Max_power,Seats,Selling_price,Record_ld_dt
0,1,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000,2025-05-10 08:45:26
1,2,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000,2025-05-10 08:45:26
2,3,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000,2025-05-10 08:45:26
3,4,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000,2025-05-10 08:45:26
4,5,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000,2025-05-10 08:45:26


In [133]:
for car in get_car_instance():
    print(car)

Idx                    int64
Car_name              object
Brand                 object
Model                 object
Vehicle_age            int64
Km_driven              int64
Seller_type           object
Fuel_type             object
Transmission_type     object
Mileage              float64
Engine                 int64
Max_power            float64
Seats                  int64
Selling_price          int64
Record_ld_dt          object
dtype: object
[11 'Hyundai Verna' 'Hyundai' 'Verna' 8 65278 'Dealer' 'Diesel' 'Manual'
 22.32 1582 126.32 5 425000 '2025-05-14 07:05:44']
{'Idx': 11, 'Car_name': 'Hyundai Verna', 'Brand': 'Hyundai', 'Model': 'Verna', 'Vehicle_age': 8, 'Km_driven': 65278, 'Seller_type': 'Dealer', 'Fuel_type': 'Diesel', 'Transmission_type': 'Manual', 'Mileage': 22.32, 'Engine': 1582, 'Max_power': 126.32, 'Seats': 5, 'Selling_price': 425000, 'Record_ld_dt': '2025-05-14 07:05:44'}
[12 'Renault Duster' 'Renault' 'Duster' 5 50000 'Individual' 'Diesel'
 'Manual' 19.64 1461 108.45 5 

In [179]:
def main(topic):
    schema_registry_conf = schema_config()
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)
    # subjects = schema_registry_client.get_subjects()
    # print(subjects)
    subject = topic+'-value'

    schema = schema_registry_client.get_latest_version(subject)
    schema_str=schema.schema.schema_str

    string_serializer = StringSerializer('utf_8')
    json_serializer = JSONSerializer(schema_str, schema_registry_client, car_to_dict)

    producer = Producer(sasl_conf())

    print("Producing user records to topic {}. ^C to exit.".format(topic))
    #while True:
        # Serve on_delivery callbacks from previous calls to produce()
    producer.poll(0.0)
    try:
        for car in get_car_instance():
            print(car)
            producer.produce(topic=topic,
                            key=string_serializer(str(uuid4())),
                            value=json_serializer(car, SerializationContext(topic, MessageField.VALUE)),
                            on_delivery=delivery_report)
    except KeyboardInterrupt:
        pass
    except ValueError:
        print("Invalid input, discarding record...")
        pass

    print("\nFlushing records...")
    producer.flush()

main("car_topic")

Producing user records to topic car_topic. ^C to exit.
{'Idx': 11, 'Car_name': 'Hyundai Verna', 'Brand': 'Hyundai', 'Model': 'Verna', 'Vehicle_age': 8, 'Km_driven': 65278, 'Seller_type': 'Dealer', 'Fuel_type': 'Diesel', 'Transmission_type': 'Manual', 'Mileage': 22.32, 'Engine': 1582, 'Max_power': 126.32, 'Seats': 5, 'Selling_price': 425000, 'Record_ld_dt': '2025-05-14 07:05:44'}
{'Idx': 12, 'Car_name': 'Renault Duster', 'Brand': 'Renault', 'Model': 'Duster', 'Vehicle_age': 5, 'Km_driven': 50000, 'Seller_type': 'Individual', 'Fuel_type': 'Diesel', 'Transmission_type': 'Manual', 'Mileage': 19.64, 'Engine': 1461, 'Max_power': 108.45, 'Seats': 5, 'Selling_price': 750000, 'Record_ld_dt': '2025-05-14 07:05:44'}
{'Idx': 13, 'Car_name': 'Mini Cooper', 'Brand': 'Mini', 'Model': 'Cooper', 'Vehicle_age': 4, 'Km_driven': 6000, 'Seller_type': 'Dealer', 'Fuel_type': 'Petrol', 'Transmission_type': 'Automatic', 'Mileage': 14.41, 'Engine': 1998, 'Max_power': 189.08, 'Seats': 5, 'Selling_price': 3250000

Kafka Consumer

In [166]:
from confluent_kafka import Consumer
from confluent_kafka.schema_registry.json_schema import JSONDeserializer

In [168]:
def main(topic):

    schema_registry_conf = schema_config()
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)
    # subjects = schema_registry_client.get_subjects()
    # print(subjects)
    subject = topic+'-value'

    schema = schema_registry_client.get_latest_version(subject)
    schema_str=schema.schema.schema_str

    json_deserializer = JSONDeserializer(schema_str,
                                         from_dict=Car.dict_to_car)

    consumer_conf = sasl_conf()
    consumer_conf.update({
                     'group.id': 'group1',
                     'auto.offset.reset': "earliest"})

    consumer = Consumer(consumer_conf)
    consumer.subscribe([topic])


    while True:
        try:
            # SIGINT can't be handled when polling, limit timeout to 1 second.
            msg = consumer.poll(1.0)
            if msg is None:
                continue

            car = json_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))

            if car is not None:
                print("User record {}: car: {}\n"
                      .format(msg.key(), car))
        except KeyboardInterrupt:
            break

    consumer.close()

main("car_topic")

In [14]:
prev_dt=df['max_record_ld_dts'].values
print(prev_dt[0])

2025-05-14T07:05:44.000000000
